In [1]:
import catalog_tools as ct
import json
import geopandas as gpd
import pandas as pd
from tqdm import tqdm
import os
from datetime import date

TARGET = "../../Data/Catalog/"

I found that filtering on the API side is not efficient

The maximimum RPRO products in one day is 58, if we don't filter on product, it takes 10 minutes to retrieve 3 years of products with the catalog API. If products are filtered it takes 2m44s to retrieve one year, so 8m15 for 3 years. Filtering is not worth it time wise.

In [2]:
def get_orbit_week(date0, date1, area_of_interest, products=["HCHO", "NO2", "SO2"], min_coverage=1, collection_id="03"):
  """
  Get the orbit for a given date and area of interest

  Parameters
  ----------
  date: str
  Date in the format "YYYY-MM-DD"
  products: list
  List of products to consider
  min_coverage: float
  Minimum coverage of the area of interest by the orbit
  collection_id: str
  Collection id of the orbit

  Returns
  -------
  output_files: list
  List of orbit filenames
  """
  # add features in flat list
  features = []
  for product in products:
    response = ct.request(date0, area_of_interest, product, date1=date1)["features"]
    assert len(response) < 100, "Went over catalog API limit, please adjust date range or area of interest."
    features.extend(response)

  response_df = gpd.GeoDataFrame.from_features(features, 
    columns=["geometry", "datetime", "s5p:type", "sat:absolute_orbit", "collection_id"])

  # add column with s3 url
  response_df["s3_url"] = [f["assets"]["data"]["href"][12:] for f in features]
  
  # remove rows with duplicate s3 url
  response_df.drop_duplicates(subset="s3_url", inplace=True)
  
  # add column with collection id
  response_df["collection_id"] = [f["id"][58:60] for f in features]
  
  # filter by collection id
  response_df = response_df[response_df["collection_id"] == collection_id]
  
  orbit_df_list = []
  # remove orbits that do not have all the products
  # iterate over unique orbits
  for orbit in response_df["sat:absolute_orbit"].unique():
    orbit_df = response_df[response_df["sat:absolute_orbit"] == orbit].copy()
    
    # check if all products are available
    products_orbit = orbit_df["s5p:type"].unique()
    if set(products).difference(products_orbit):
      continue
    
    # calculate coverage and ignore if below threshold
    orbit_df.loc[:, "coverage"] = orbit_df.apply(lambda row: ct.calculate_coverage(row["geometry"], area_of_interest), axis=1)
    if orbit_df['coverage'].min() < min_coverage:
      continue
        
    orbit_df_list.append(orbit_df)
    
  if not orbit_df_list:
    # return empty dataframe if no orbits are found
    return pd.DataFrame()
  
  response_df = pd.concat(orbit_df_list)        
  response_df.set_index(["sat:absolute_orbit", "s5p:type"], inplace=True)
  response_df.sort_index(inplace=True)
  return response_df

It takes about 5 minutes per region

In [3]:

# open ../../Data/aoi.json as dict
with open("../../Data/aoi.json", "r") as f:
  aoi = json.load(f)

# iterate over the areas of interest
for aoi_name, area_of_interest in aoi.items():
  # check if the download list already exists
  if os.path.exists(f"{TARGET}/{aoi_name}/download_list.csv"):
    print(f"Download list for {aoi_name} already exists, skipping...")
    continue
  
  # info on date range: https://www.temis.nl/airpollution//no2col/tropomi_no2_data_versions.php
  date0 = "2018-05-01"
  date_min1 = "2022-07-25"
  download_list = []
  # empty_count = 0
  for date1 in tqdm(pd.date_range(start=date0, end=date_min1, freq="2W"), 
                    desc=f"Downloading {aoi_name} file list for box {area_of_interest}", unit="2 weeks", colour="green"):
    date1 = date1.date()
    # Get the dataframe for the specific date range, you can change the products and collection_id
    response = get_orbit_week(date0, date1, area_of_interest)
    if not response.empty:
      download_list.append(response)
    # elif empty_count > 3:
    #   print(f"No data found for {aoi_name} for the last 2 months, skipping...")
    #   break
    # else:
    #   empty_count += 1
    date0 = date1
    
  if not download_list:
    print(f"No data found for {aoi_name}, skipping...")
    continue
  download_list = pd.concat(download_list)

  # remove geometry and collection id column
  download_list.drop(columns=["geometry", "collection_id"], inplace=True)
  
  # remove duplicates 
  download_list.drop_duplicates(subset=["s3_url"], inplace=True)

  # save to csv
  if not os.path.exists(f"{TARGET}/{aoi_name}"):
    os.makedirs(f"{TARGET}/{aoi_name}")
  download_list.to_csv(f"{TARGET}/{aoi_name}/download_list.csv")

Example:


| geometry | datetime | s5p:type | sat:absolute_orbit | collection_id | s3_url | coverage |
|---|---|---|---|---|---|---|
| 14 MULTIPOLYGON (((-180.00000 -90.00000, 180.0000... | 2019-11-28T11:40:16Z | HCHO | 11011 | 3 | Sentinel-5P/TROPOMI/L2__HCHO__/2019/11/28/S5P_... | 1.000000 |
| 35 MULTIPOLYGON (((-180.00000 -90.00000, 180.0000... | 2019-11-28T11:40:16Z | NO2 | 11011 | 3 | Sentinel-5P/TROPOMI/L2__NO2___/2019/11/28/S5P_... | 0.999997 |
| 56 MULTIPOLYGON (((-180.00000 -90.00000, 180.0000... | 2019-11-28T11:40:16Z | SO2 | 11011 | 3 | Sentinel-5P/TROPOMI/L2__SO2___/2019/11/28/S5P_... | 1.000000 |

Absolute orbits can have ever so slightly different geometries, by filtering on coverage per row, we will end up with some files that have missing products. This is an issue with Catalog_1.0 and EODATA_1.0, but is fixed in 1.1